In [46]:
import pandas as pd
import numpy as np
from pybaseball import statcast_batter
from pybaseball import playerid_lookup
from pybaseball import cache

pd.get_option("display.max_columns")
cache.enable()

In [47]:
# we want key_mlbam
bobby = playerid_lookup("Bradley", "Bobby")
print(bobby)

  name_last name_first  key_mlbam key_retro  key_bbref  key_fangraphs  \
0   bradley      bobby     656252  bradb002  bradlbo01          17278   

   mlb_played_first  mlb_played_last  
0            2019.0           2021.0  


In [48]:
# get pitch by pitch data for bobby, just in case
full_data = statcast_batter('2021-04-01', '2021-09-01', player_id = 656252)
#print(full_data.columns.values.tolist())

In [49]:
# extract hit data from Bobby
hit_dataframe = pd.DataFrame(full_data, columns = ["pitch_type", "game_date", "events", "description", "estimated_ba_using_speedangle", "estimated_woba_using_speedangle", "babip_value"])
# we only want balls that Bobby hit into play
contains_values_hit = hit_dataframe.loc[(hit_dataframe['description'] == "hit_into_play")]
# reset index count
contains_values_hit.reset_index(drop=True, inplace=True)
#print(contains_values)
hit_data = open("hit_data.txt", "w")
hit_data.write(contains_values_hit.to_string())
hit_data.close()

In [50]:
strikeout_dataframe = pd.DataFrame(full_data, columns = ["pitch_type", "game_date", "events", "description", "zone", "balls", "strikes", "release_spin_rate"])
contains_values_strikeout = strikeout_dataframe.loc[(strikeout_dataframe['events'] == "strikeout")]
contains_values_strikeout.reset_index(drop=True, inplace=True)
#print(contains_values_strikeout)
strikeout_data = open("strikeout_data.txt", "w")
strikeout_data.write(contains_values_strikeout.to_string())
strikeout_data.close()

In [51]:
shift_dataframe = pd.DataFrame(full_data, columns = ["if_fielding_alignment", "of_fielding_alignment"])
#print(shift_data)

shift_data = open("shift_data.txt", "w")
shift_data.write(shift_dataframe.to_string())
shift_data.close()

# Analyzing the data we have extracted
Let's count his most common strikeout pitches

Here's a little glossary for pitch abbreviations
FA = fastball
FF = four-seam fastball
FT = two-seam fastball
FC = fastball (cutter)
FS / SI / SF = fastball (sinker, split-fingered)
SL = slider
CH = changeup
CB / CU = curveball
KC = knuckle-curve
KN = knuckleball
EP = eephus
UN / XX = unidentified
PO / FO = pitch out

In [52]:
contains_values_strikeout["pitch_type"].value_counts()


FF    15
CH    13
SL    13
SI    10
FC     6
CU     5
FS     4
KC     1
Name: pitch_type, dtype: int64

Fastballs: 35 pitches
Offspeed: 32 pitches

It doesn't appear as if pitch type alone is something that causes Bobby to strike out more or less this year

In [53]:
contains_values_strikeout["zone"].value_counts()


14    12
11     9
7      8
13     7
9      6
1      5
2      4
4      4
12     4
3      3
8      3
6      1
5      1
Name: zone, dtype: int64

Here is our possible culprit, pitch location!

Nearly a whopping 60% (59.7%) of his strikeouts come from the zones outside of the called strike zone.

Now, obviously good pitchers will get you to swing outside of the called zone, but better plate discipline is certainly something that can be taught and developed.